# 预先进行聚类


In [2]:
import os
import pandas as pd
import torch
from tqdm import tqdm
from cuml.cluster import KMeans
import cupy as cp

# from sklearn.cluster import KMeans

# Your existing setup
input_path = '/data/wsi/TCTGC50k-features/gigapath-coarse/pt'
input_label = '/data/wsi/TCTGC50k-labels/6_labels/TCTGC50k-v15-train.csv'
labels = pd.read_csv(input_label)
n_clusters = 5
output_path = './cluster'

# Create DataFrame with WSI names
df = labels[['wsi_name']].copy()
df['cluster_label'] = None  # Add column for storing pooled features

# Process each WSI
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing WSIs"):
    wsi_name = row['wsi_name']
    pt_path = os.path.join(input_path, f"{wsi_name}.pt")
    
    try:
        features = torch.load(pt_path)
        # Load features
        # kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        # cluster_labels = kmeans.fit_predict(features)

        patch_features_cp = cp.asarray(features)
        # patch_features_cp = patch_features
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(patch_features_cp)
        df.at[idx, 'cluster_label'] = " ".join(map(str, cluster_labels))
        
    except Exception as e:
        print(f"Error processing {wsi_name}: {str(e)}")
        df.at[idx, 'cluster_label'] = None  # Mark as failed

# Optional: Save the DataFrame with pooled features
output_df_path = os.path.join(output_path, f"kmeans_{n_clusters}.csv")
df.to_csv(output_df_path, index=False)
print(f"Saved pooled features to {output_df_path}")

Processing WSIs:   0%|          | 0/38339 [00:00<?, ?it/s]/tmp/ipykernel_2983/774604183.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  features = torch.load(pt_path)
P

Saved pooled features to ./cluster/kmeans_5.csv
